In [ ]:
#| default_exp misc.fc_decomposer

In [ ]:
#| include: false
from nbdev.showdoc import *

%config InlineBackend.figure_format = 'retina'

We can factorize our big fully-connected layers and replace them by an approximation of two smaller layers. The idea is to make an SVD decomposition of the weight matrix, which will express the original matrix in a product of 3 matrices: $U \Sigma V^T$
With $\Sigma$ being a diagonal matrix with non-negative values along its diagonal (the singular values). We then define a value $k$ of singular values to keep and modify matrices $U$ and $V^T$ accordingly. The resulting will be an approximation of the initial matrix.

![](imgs/svd.png "SVD Decomposition")

In [ ]:
#| export
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy

/Users/nathan/opt/miniconda3/envs/nbdev/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export
class FC_Decomposer:

    def __init__(self):
        super().__init__()
        
    def decompose(self, model, percent_removed=0.5):

        new_model = copy.deepcopy(model)

        module_names = list(new_model._modules)

        for k, name in enumerate(module_names):

            if len(list(new_model._modules[name]._modules)) > 0:
                new_model._modules[name] = self.decompose(new_model._modules[name], percent_removed)

            else:
                if isinstance(new_model._modules[name], nn.Linear):
                    # Folded BN
                    layer = self.SVD(new_model._modules[name], percent_removed)

                    # Replace old weight values
                    new_model._modules[name] = layer # Replace the FC Layer by the decomposed version
        return new_model


    def SVD(self, layer, percent_removed):

        W = layer.weight.data
        U, S, V = torch.svd(W)
        L = int((1.-percent_removed)*U.shape[0])
        W1 = U[:,:L]
        W2 = torch.diag(S[:L]) @ V[:,:L].t()
        layer_1 = nn.Linear(in_features=layer.in_features, 
                    out_features=L, bias=False)
        layer_1.weight.data = W2

        layer_2 = nn.Linear(in_features=L, 
                    out_features=layer.out_features, bias=True)
        layer_2.weight.data = W1

        if layer.bias.data is None: 
            layer_2.bias.data = torch.zeros(*layer.out_features.shape)
        else:
            layer_2.bias.data = layer.bias.data

        return nn.Sequential(layer_1, layer_2)

In [ ]:
show_doc(FC_Decomposer.decompose)

---

### FC_Decomposer.decompose

>      FC_Decomposer.decompose (model, percent_removed=0.5)

A tutorial about how to use the `FC_Decomposer` functionalities can be found [here](https://nathanhubens.github.io/fasterai/tutorial.fc_decomposer.html)